# Aggregration of Experiment Results

This notebook pulls results from the `output` folder and aggregates them in a large table that can than be formated, pivoted etc.

In [7]:
from pathlib import Path

OUTPUT_PATH = Path("/work/smt4/thulke/vieweg/SER/Code/sisyphus/output")

## Metrics

In [8]:
import pandas as pd
import glob

dfs = []
for path in map(Path, glob.glob(f"{OUTPUT_PATH}/**/metrics.csv", recursive=True)):
    if "pp" in path.parts:
        continue

    print(path)
    metrics_df = pd.read_csv(path)
    metrics_df["experiment"] = path.parts[-3]
    metrics_df["dataset"] = path.parts[-4]
    dfs.append(metrics_df)

metrics = pd.concat(dfs, axis=0, ignore_index=True)
metrics.head(20)

/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_finetune_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_featureextract_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/yuan/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_featureextract_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_featureextract_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/yuan/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_finetune_facebook_wav2vec2-ba

,split,metric,value,experiment,dataset
0,train,eer,0.821296,lj_finetune_facebook_wav2vec2-base-960h,rav
1,val,eer,0.700000,lj_finetune_facebook_wav2vec2-base-960h,rav
2,test,eer,0.823333,lj_finetune_facebook_wav2vec2-base-960h,rav
3,train,beer_Happiness,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
4,train,beer_Anger,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
5,train,beer_Sadness,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
6,train,beer_Fear,0.777778,lj_finetune_facebook_wav2vec2-base-960h,rav
7,train,beer_Disgust,0.798611,lj_finetune_facebook_wav2vec2-base-960h,rav
8,train,beer_Surprise,1.000000,lj_finetune_facebook_wav2vec2-base-960h,rav
9,train,beer_Calmness,0.083333,lj_finetune_facebook_wav2vec2-base-960h,rav


In [9]:
from IPython.display import Markdown

for split in metrics.split.unique():
    # global metrics
    global_only_metrics = metrics.loc[
        ~metrics.metric.str.contains("_") ^ metrics.metric.str.contains("total")
    ]
    global_only_metrics = global_only_metrics.assign(
        metric=global_only_metrics.metric.apply(lambda s: s.split("_")[0])
    )

    global_pivot_metrics = (
        global_only_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "metric"], values="value"
        )
        * 100
    )

    if split == "test":
        global_pivot_metrics.loc["lj_finetune_lit", ("rav", "eer")] = 18.01
        global_pivot_metrics.loc["lj_featureextract_lit", ("rav", "eer")] = 43.47
        global_pivot_metrics.loc["yuan_lit", ("iem", "beer")] = 100 - 75.6
        global_pivot_metrics.loc["yuan_lit", ("iem", "eer")] = 100 - 75.5

    # beer per emo
    per_emo_metrics = metrics.loc[
        metrics.metric.str.contains("_") & ~metrics.metric.str.contains("total")
    ]
    per_emo_metrics = per_emo_metrics.assign(
        emotion=per_emo_metrics.metric.apply(lambda s: s.split("_")[-1])
    )

    per_emo_pivot_metrics = (
        per_emo_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "emotion"], values="value"
        )
        * 100
    )

    # print
    display(Markdown(f"# {split}"))
    display(global_pivot_metrics.round(2).fillna("--"))
    display(per_emo_pivot_metrics.round(2).fillna("--"))

# train

dataset                                               rav                \
metric                                                eer   beer    per   
experiment                                                                
lj_featureextract_facebook_wav2vec2-base-960h       33.15  35.07     --   
lj_featureextract_jonatasgrosman_wav2vec2-large...  38.89  39.32     --   
lj_finetune_facebook_wav2vec2-base-960h             82.13  83.25     --   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   5.19   4.95     --   
yuan                                                88.15  88.28  95.77   

dataset                                               iem                
metric                                                eer   beer    per  
experiment                                                               
lj_featureextract_facebook_wav2vec2-base-960h       16.57  17.45     --  
lj_featureextract_jonatasgrosman_wav2vec2-large...  29.30  29.42     --  
lj_finetune_facebook_wav2vec2-base-960h             80.58  75.00     --  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  47.67  50.62     --  
yuan                                                74.09  74.21  93.34

dataset                                                  rav                  \
emotion                                            Happiness   Anger Sadness   
experiment                                                                     
lj_featureextract_facebook_wav2vec2-base-960h          53.47   20.83   44.44   
lj_featureextract_jonatasgrosman_wav2vec2-large...     29.86   20.83   58.33   
lj_finetune_facebook_wav2vec2-base-960h               100.00  100.00  100.00   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     22.22    7.64    2.78   
yuan                                                   88.89   88.19   90.28   

dataset                                                                     \
emotion                                              Fear Disgust Surprise   
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h       28.47   25.69    24.31   
lj_featureextract_jonatasgrosman_wav2vec2-large...  38.89   47.92    27.08   
lj_finetune_facebook_wav2vec2-base-960h             77.78   79.86   100.00   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   0.00    0.00     0.00   
yuan                                                91.67   87.50    83.33   

dataset                                                                   iem  \
emotion                                            Calmness Neutral Happiness   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h         19.44   63.89     17.39   
lj_featureextract_jonatasgrosman_wav2vec2-large...    45.83   45.83     45.19   
lj_finetune_facebook_wav2vec2-base-960h                8.33  100.00    100.00   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     5.56    1.39     44.60   
yuan                                                  86.11   90.28     72.67   

dataset                                                                     
emotion                                              Anger Sadness Neutral  
experiment                                                                  
lj_featureextract_facebook_wav2vec2-base-960h         9.49   12.24   30.67  
lj_featureextract_jonatasgrosman_wav2vec2-large...   15.49   17.87   39.15  
lj_finetune_facebook_wav2vec2-base-960h             100.00  100.00    0.00  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   31.55   26.34  100.00  
yuan                                                 74.18   72.15   77.82

# val

dataset                                               rav                \
metric                                                eer   beer    per   
experiment                                                                
lj_featureextract_facebook_wav2vec2-base-960h       48.33  51.56     --   
lj_featureextract_jonatasgrosman_wav2vec2-large...  45.00  43.75     --   
lj_finetune_facebook_wav2vec2-base-960h             70.00  71.88     --   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  21.67  21.88     --   
yuan                                                90.00  90.62  95.16   

dataset                                               iem                
metric                                                eer   beer    per  
experiment                                                               
lj_featureextract_facebook_wav2vec2-base-960h       36.36  37.07     --  
lj_featureextract_jonatasgrosman_wav2vec2-large...  44.19  43.84     --  
lj_finetune_facebook_wav2vec2-base-960h             74.05  75.00     --  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  51.15  50.44     --  
yuan                                                76.92  77.32  93.04

dataset                                                  rav                 \
emotion                                            Happiness  Anger Sadness   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h           87.5   12.5    37.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...      75.0    0.0    50.0   
lj_finetune_facebook_wav2vec2-base-960h                100.0  100.0   100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...      50.0   25.0    25.0   
yuan                                                    87.5  100.0    87.5   

dataset                                                                    \
emotion                                             Fear Disgust Surprise   
experiment                                                                  
lj_featureextract_facebook_wav2vec2-base-960h       12.5    37.5     87.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...  75.0    25.0     87.5   
lj_finetune_facebook_wav2vec2-base-960h             50.0    25.0    100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  12.5     0.0     25.0   
yuan                                                87.5    87.5     87.5   

dataset                                                                   iem  \
emotion                                            Calmness Neutral Happiness   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h          37.5   100.0     50.00   
lj_featureextract_jonatasgrosman_wav2vec2-large...     12.5    25.0     57.89   
lj_finetune_facebook_wav2vec2-base-960h                 0.0   100.0    100.00   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     12.5    25.0     44.83   
yuan                                                   87.5   100.0     82.22   

dataset                                                                     
emotion                                              Anger Sadness Neutral  
experiment                                                                  
lj_featureextract_facebook_wav2vec2-base-960h        14.29   24.00   60.00  
lj_featureextract_jonatasgrosman_wav2vec2-large...   30.56   33.33   53.57  
lj_finetune_facebook_wav2vec2-base-960h             100.00  100.00    0.00  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   30.61   26.32  100.00  
yuan                                                 63.89   84.00   79.17

# test

dataset                                               rav                \
metric                                                eer   beer    per   
experiment                                                                
lj_featureextract_facebook_wav2vec2-base-960h       53.67   55.0     --   
lj_featureextract_jonatasgrosman_wav2vec2-large...  55.33  55.62     --   
lj_finetune_facebook_wav2vec2-base-960h             82.33  83.44     --   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  26.67  25.94     --   
yuan                                                 84.0  84.38  95.86   
lj_finetune_lit                                     18.01     --     --   
lj_featureextract_lit                               43.47     --     --   
yuan_lit                                               --     --     --   

dataset                                               iem                
metric                                                eer   beer    per  
experiment                                                               
lj_featureextract_facebook_wav2vec2-base-960h       46.58  47.85     --  
lj_featureextract_jonatasgrosman_wav2vec2-large...  50.81  50.68     --  
lj_finetune_facebook_wav2vec2-base-960h             86.64   75.0     --  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  42.67   49.8     --  
yuan                                                75.24  74.68  90.64  
lj_finetune_lit                                        --     --     --  
lj_featureextract_lit                                  --     --     --  
yuan_lit                                             24.5   24.4     --

dataset                                                  rav                 \
emotion                                            Happiness  Anger Sadness   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h           70.0   42.5    57.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...      42.5   30.0    72.5   
lj_finetune_facebook_wav2vec2-base-960h                100.0  100.0   100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...      62.5   17.5    45.0   
yuan                                                    92.5   87.5    90.0   

dataset                                                                    \
emotion                                             Fear Disgust Surprise   
experiment                                                                  
lj_featureextract_facebook_wav2vec2-base-960h       62.5    62.5     47.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...  70.0    70.0     47.5   
lj_finetune_facebook_wav2vec2-base-960h             70.0    82.5    100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   2.5    15.0     17.5   
yuan                                                82.5    82.5     77.5   

dataset                                                                   iem  \
emotion                                            Calmness Neutral Happiness   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h          22.5    75.0     48.70   
lj_featureextract_jonatasgrosman_wav2vec2-large...     52.5    60.0     60.00   
lj_finetune_facebook_wav2vec2-base-960h                15.0   100.0    100.00   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     32.5    15.0     38.26   
yuan                                                   72.5    90.0     81.74   

dataset                                                                     
emotion                                              Anger Sadness Neutral  
experiment                                                                  
lj_featureextract_facebook_wav2vec2-base-960h        23.38   60.81   58.54  
lj_featureextract_jonatasgrosman_wav2vec2-large...   23.38   60.81   58.54  
lj_finetune_facebook_wav2vec2-base-960h             100.00  100.00    0.00  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   29.87   31.08  100.00  
yuan                                                 72.73   66.22   78.05